## 說明：
總共要產生三種訓練集
- 訓練集
- 驗證集
- 測試集   
其中驗證集及測試集要經二階段建立。

In [1]:
import sys
import os
import subprocess
import glob
import numpy as np
import wavio
import time;
import random;
sys.path.append(os.path.abspath("../"))
import re

In [2]:
from common.tlopts import display_info;
import common.utils as U;
from Libs.SharedLibs import *

In [3]:
import argparse

In [4]:
from datetime import datetime

## Global Shared Variables

### define sounds sources folders

In [5]:
training_positive_sounds_src = "../datasets/forOneClassModel_alarm/train/alarm/"
training_negative_sounds_src = "../datasets/forOneClassModel_alarm/train/other_sounds/20K_25Classes_160Items/"
val_positive_sounds_src = "../datasets/forOneClassModel_alarm/test_val/alarm/"
val_negative_sounds_src = "../datasets/forOneClassModel_alarm/test_val/other_sounds/"

### define save pathes

In [6]:
training_npz = "../datasets/forOneClassModel_alarm/train/trainSet_{}.npz"
val_src_npz = "../datasets/forOneClassModel_alarm/test_val/src_npz/val_src.npz"
final_val_npz = "../datasets/forOneClassModel_alarm/test_val/final_val_test_npz/final_valSet_{}.npz"

## Utility Functions

In [7]:
def getFolderList(rootDir=None, recursive=False):
    if not recursive:
        return next(os.walk(rootDir));
    else:
        return [x[0] for x in os.walk(rootDir)]

In [8]:
# generate opt files for data preparation
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='ACDNet_TL_Model_Extend',  required=False);
    parser.add_argument('--data', default='../datasets/processed/',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['10']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args();

    """
    current best setting for accuracy: 96.5
    opt.batchSize = 32;
    opt.LR = 0.1;
    opt.weightDecay = 5e-3;
    opt.momentum = 0.09;
    opt.schedule = [0.3, 0.5, 0.9];
    """
    #Leqarning settings
    opt.batchSize = 32;
    opt.LR = 0.1;
    opt.weightDecay = 5e-3;#1e-2;#5e-3;#5e-4;
    opt.momentum = 0.09;
    opt.nEpochs = 800;
    opt.schedule = [0.3, 0.5, 0.9];
    opt.warmup = 10;
    if torch.backends.mps.is_available():
        opt.device="mps"; #for apple m2 gpu
    elif torch.cuda.is_available():
        opt.device="cuda:0"; #for nVidia gpu
    else:
        opt.device="cpu"
    print(f"***Use device:{opt.device}");
    # opt.device = torch.device("cuda:0" if  else "cpu");
    #Basic Net Settings
    opt.nClasses = 2#50;
    opt.nFolds = 1;
    opt.splits = [i for i in range(1, opt.nFolds + 1)];
    opt.sr = 20000;
    opt.inputLength = 30225;
    #Test data
    opt.nCrops = 2;
    opt.TLAcdnetConfig = [8,64,32,64,64,128,128,256,256,512,512,2];
    return opt

In [9]:
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

In [10]:
def testGetSoundsAndLabels(srcDir):
    wav_list = []
    wav_list = getFileList(srcDir)
    return wav_list

In [14]:
print(f"get all positive wavs in {training_positive_sounds_src}:\n total counts:{len(testGetSoundsAndLabels(training_positive_sounds_src))}")
print(f"get all negative wavs in {training_negative_sounds_src}:\n total counts:{len(testGetSoundsAndLabels(training_negative_sounds_src))}")

get all positive wavs in ../datasets/forOneClassModel_alarm/train/alarm/:
 total counts:160
get all negative wavs in ../datasets/forOneClassModel_alarm/train/other_sounds/20K_25Classes_160Items/:
 total counts:157


In [24]:
def create_dataset(src_path, dst_path, classes_dict):
    # print('* {} -> {}'.format(src_path, dst_path))
    my_dataset = {};
    print(f"--Start to preparing training dataset...---------------");
    my_dataset['fold1'] = {}
    my_sounds = []
    my_labels = []
    wav_list = None
    for k in classes_dict:
        if k == 52:
            cur_src_dir = os.path.join(src_path,classes_dict[k]);
            wav_list = getFileList(cur_src_dir);
        elif k == 99:
            cur_src_dir = os.path.join(src_path,classes_dict[k]);
            wav_list = getFileList(cur_src_dir);
        # print(wav_list)
        # print(f"current source directory:{cur_src_dir}");
        for wav_file in wav_list:
            sound = wavio.read(wav_file).data.T[0]
            start = sound.nonzero()[0].min()
            end = sound.nonzero()[0].max()
            sound = sound[start: end + 1]  # Remove silent sections
            label = int(k);#int(os.path.splitext(wav_file)[0].split('-')[-1])
            my_sounds.append(sound)
            my_labels.append(k)
            print(f"sound:{os.path.basename(wav_file)} is chopped:\n   lable:{k}\n   from {start} to {end} \n   len:{(end-start)/20000}\n");
            # print(f"sound:{wav_file}\nlabel:{k}") 
    print(f"--End of preparing training dataset-------------------");

    my_dataset['fold1']['sounds'] = my_sounds
    my_dataset['fold1']['labels'] = my_labels
    npzname = dst_path.format(genDataTimeStr());
    np.savez(npzname, **my_dataset)
    print(f"npz file:{npzname}")a

SyntaxError: invalid syntax (610949817.py, line 34)